In [1]:
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import trt_pose.coco
import math
import os
import numpy as np
import traitlets
import sys


In [35]:
with open('preprocess/hand_pose.json', 'r') as f:
    hand_pose = json.load(f)
topology = trt_pose.coco.coco_category_to_topology(hand_pose)
print(hand_pose)
print(topology)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

print(num_parts)
print(num_links)

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
print(model)

{'supercategory': 'hand', 'id': 1, 'name': 'hand', 'keypoints': ['palm', 'thumb_1', 'thumb_2', 'thumb_3', 'thumb_4', 'index_finger_1', 'index_finger_2', 'index_finger_3', 'index_finger_4', 'middle_finger_1', 'middle_finger_2', 'middle_finger_3', 'middle_finger_4', 'ring_finger_1', 'ring_finger_2', 'ring_finger_3', 'ring_finger_4', 'baby_finger_1', 'baby_finger_2', 'baby_finger_3', 'baby_finger_4'], 'skeleton': [[1, 5], [1, 9], [1, 13], [1, 17], [1, 21], [2, 3], [3, 4], [4, 5], [6, 7], [7, 8], [8, 9], [10, 11], [11, 12], [12, 13], [14, 15], [15, 16], [16, 17], [18, 19], [19, 20], [20, 21]]}
tensor([[ 0,  1,  0,  4],
        [ 2,  3,  0,  8],
        [ 4,  5,  0, 12],
        [ 6,  7,  0, 16],
        [ 8,  9,  0, 20],
        [10, 11,  1,  2],
        [12, 13,  2,  3],
        [14, 15,  3,  4],
        [16, 17,  5,  6],
        [18, 19,  6,  7],
        [20, 21,  7,  8],
        [22, 23,  9, 10],
        [24, 25, 10, 11],
        [26, 27, 11, 12],
        [28, 29, 13, 14],
        [30, 

In [2]:
with open('preprocess/hand_pose.json', 'r') as f:
    hand_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(hand_pose)
print(topology)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
import torch


WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()
# print(data)

if not os.path.exists('model/hand_pose_resnet18_att_244_244_trt.pth'):
    MODEL_WEIGHTS = 'model/hand_pose_resnet18_att_244_244.pth'
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    import torch2trt
    model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    OPTIMIZED_MODEL = 'model/hand_pose_resnet18_att_244_244_trt.pth'
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)


OPTIMIZED_MODEL = 'model/hand_pose_resnet18_att_244_244_trt.pth'
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

tensor([[ 0,  1,  0,  4],
        [ 2,  3,  0,  8],
        [ 4,  5,  0, 12],
        [ 6,  7,  0, 16],
        [ 8,  9,  0, 20],
        [10, 11,  1,  2],
        [12, 13,  2,  3],
        [14, 15,  3,  4],
        [16, 17,  5,  6],
        [18, 19,  6,  7],
        [20, 21,  7,  8],
        [22, 23,  9, 10],
        [24, 25, 10, 11],
        [26, 27, 11, 12],
        [28, 29, 13, 14],
        [30, 31, 14, 15],
        [32, 33, 15, 16],
        [34, 35, 17, 18],
        [36, 37, 18, 19],
        [38, 39, 19, 20]], dtype=torch.int32)


<All keys matched successfully>

In [3]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology,cmap_threshold=0.15, link_threshold=0.15)
draw_objects = DrawObjects(topology)

In [8]:
def create_directories_for_classes(no_of_classes, path_dir, dataset_name):
    dir_ = os.path.join(path_dir, dataset_name)
    for i in range(no_of_classes):
        dir_to_create = os.path.join(dir_,"%s" % (i+1))
        try:
            os.makedirs(dir_to_create)
        except FileExistsError:
            
            print(os.path.join("The following directory was not created because it already exsists", dir_ , ))


In [9]:
dir_datasets = '/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/' #give the path to where you want to save you collected data
dataset_name = "golf_dataset_train" #change this to hand_dataset_test when you are collecting data for test
no_of_classes = 6
create_directories_for_classes(no_of_classes, dir_datasets, dataset_name )

/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train
/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train
/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train
/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train
/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train
/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train


In [6]:
import ipywidgets.widgets as widgets
dir_ = os.path.join(dir_datasets, dataset_name)
curr_class_no = 1
button_layout = widgets.Layout(width='128px', height='32px')
curr_dir = os.path.join(dir_,'%s'%curr_class_no )
collecting_button = widgets.Button(description= 'Collect Class ' + str(curr_class_no), button_style='success', layout=button_layout)
prev_button = widgets.Button(description='Previous Class', button_style='primary', layout=button_layout)
nxt_button = widgets.Button(description='Next Class', button_style='info', layout=button_layout)

dir_count = widgets.IntText(layout=button_layout, value=len(os.listdir(curr_dir)))
dir_count.continuous_update

False

In [7]:
from uuid import uuid1
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_s.value)
def save_dir():
    global curr_dir, dir_count
    save_snapshot(curr_dir)
    dir_count.value = len(os.listdir(curr_dir))
def prev_dir():
    global curr_class_no, curr_dir, no_of_classes
    if curr_class_no>1:
        curr_class_no-=1
    curr_dir = os.path.join(dir_,'%s'%curr_class_no )
    collecting_button.description = 'Collect Class ' + str(curr_class_no)
    dir_count.value = len(os.listdir(curr_dir))
    dir_count.continuous_update
def nxt_dir():
    global curr_class_no, curr_dir, no_of_classes
    if curr_class_no<no_of_classes:
        curr_class_no+=1
    curr_dir = os.path.join(dir_,'%s'%curr_class_no )
    collecting_button.description = 'Collect Class ' + str(curr_class_no)
    dir_count.value = len(os.listdir(curr_dir))

        

collecting_button.on_click(lambda x: save_dir())
nxt_button.on_click(lambda x: nxt_dir())
prev_button.on_click(lambda x: prev_dir())

In [8]:

import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [9]:
from preprocessdata import preprocessdata
preprocessdata = preprocessdata(topology, num_parts)
from gesture_classifier import gesture_classifier
gesture_classifier = gesture_classifier()

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [10]:
def draw_joints(image, joints):
    count = 0
    for i in joints:
        if i==[0,0]:
            count+=1
    if count>= 3:
        return 
    for i in joints:
        cv2.circle(image, (i[0],i[1]), 2, (0,0,255), 1)
    cv2.circle(image, (joints[0][0],joints[0][1]), 2, (255,0,255), 1)
    for i in hand_pose['skeleton']:
        if joints[i[0]-1][0]==0 or joints[i[1]-1][0] == 0:
            break
        cv2.line(image, (joints[i[0]-1][0],joints[i[0]-1][1]), (joints[i[1]-1][0],joints[i[1]-1][1]), (0,255,0), 1)

In [11]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
print(WIDTH)
print(HEIGHT)

224
224


In [12]:
# from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

# camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=1)
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

In [13]:
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

RuntimeError: Could not initialize camera.  Please see error trace.

In [14]:
import ipywidgets
from IPython.display import display


image_w = ipywidgets.Image(format='jpeg', width=224, height=224)
image_s = ipywidgets.Image(format='jpeg', width=224, height=224)


In [39]:
display(image_w)
display(widgets.HBox([dir_count, collecting_button]))
display(widgets.HBox([ nxt_button]))
display(widgets.HBox([ prev_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [16]:
def execute(change):
    image = change['new']
    image_s.value = bgr8_to_jpeg(image[:, ::-1, :])
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    joints = preprocessdata.joints_inference(image, counts, objects, peaks)
    draw_joints(image, joints)
    #draw_objects(image, counts, objects, peaks)# try this for multiple hand pose prediction 
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])

In [17]:
execute({'new': camera.value})

In [40]:
camera.observe(execute, names='value')

In [41]:
camera.unobserve_all()

In [26]:
camera.running = False

In [19]:
def generate_labels(dir_, dataset_name):
    labels = []
    starting_label = 1
    for i in range(len(os.listdir(dir_))):
        dir_to_check = os.path.join(dir_,"%s" % (i+1))
        for j in range(len(os.listdir(dir_to_check))):
            labels.append(starting_label)
        starting_label+=1
    labels_to_dict = {"labels": labels}
    with open((dir_+'.json'), 'w') as outfile:
        json.dump(labels_to_dict, outfile)
    return labels      

In [23]:
def rename_images(dir_):
    overall_count = 0
    #dir_ = dir_+dataset_name
    for i in range(len(os.listdir(dir_))):
        dir_to_check = os.path.join(dir_,"%s" % (i+1))
        dir_to_check+='/'
        for count, filename in enumerate(os.listdir(dir_to_check)):
            dst = "%08d.jpg"% overall_count
            src = dir_to_check+filename
            dst = dir_to_check+dst 
            os.rename(src, dst)
            overall_count+=1

In [24]:
generate_labels(dir_, dataset_name)
rename_images(dir_)

FileNotFoundError: [Errno 2] No such file or directory: '/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train/7'

In [25]:
import shutil
dir_training = dir_datasets +'/training/'#change this to /test/ when you are collecting data for test
try:
    os.makedirs(dir_training)
except FileExistsError:
    print(os.path.join("The following directory was not created because it already exsists", dir_ , ))
for i in range(len(os.listdir(dir_))):
    dir_to_check = os.path.join(dir_,"%s" % (i+1))+'/'
    for count, filename in enumerate(os.listdir(dir_to_check)):
            src = dir_to_check+filename
            shutil.move(src,dir_training)
    os.rmdir(dir_to_check)
shutil.move(dir_training,dir_)
shutil.move(dir_+'.json',dir_)

FileNotFoundError: [Errno 2] No such file or directory: '/media/wes-237b/JetsonTX2/Golf-Vision/data_collection/golf_dataset_train/7/'